In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
import nltk

In [9]:
stopwords = stopwords.words("english")

In [10]:
def get_stats(group):
    return {'count': group.count()}

def grouped_data_stats(df, col_name):
    dataset = df[col_name].groupby(df[col_name])
    stats = dataset.apply(get_stats).unstack()
    return stats.sort(['count'], ascending=[0])

In [11]:
data = pd.read_csv("/Users/rituc/twitter_data/stream_table1.csv")

In [12]:
len(data)

65499

In [13]:
data.columns

Index([u'Tweet Id', u'Date', u'Hour', u'User Name', u'Nickname', u'Bio',
       u'Tweet content', u'Favs', u'RTs', u'Latitude', u'Longitude',
       u'Country', u'Place (as appears on Bio)', u'Profile picture',
       u'Followers', u'Following', u'Listed', u'Tweet language (ISO 639-1)',
       u'Tweet Url'],
      dtype='object')

In [15]:
# data

In [21]:
bio = list(data["Bio"])

In [23]:
# bio

In [24]:
grouped_data_stats(data, "User Name")

/Users/rituc/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,count
User Name,
511 New York,403
Winn-Dixie Careers,280
CVS Health Jobs,241
SONIC Jobs,198
SHC Careers,176
Vitamin Shoppe Jobs,172
Favorite Jobs,158
DineHere.us,152
CompassUSAJobBoard,147


In [25]:
tweets = data["Tweet content"]

In [15]:
def tokenize_and_stem(text):
#     text = unicode(text, errors='replace')
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text.decode('utf-8')) for word in nltk.word_tokenize(sent.decode('utf-8'))]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [13]:
import sys  
import re
reload(sys)  
sys.setdefaultencoding('utf8')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import codecs

In [14]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in tweets:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
#     allwords_tokenized = tokenize_only(i)
#     totalvocab_tokenized.extend(allwords_tokenized)

In [16]:
vocab_frame = pd.DataFrame({'words': totalvocab_stemmed}, index = totalvocab_stemmed)

In [17]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.8, stop_words=stopwords, max_features=20000)

In [18]:
tfidf_matrix = tfidf_vectorizer.fit_transform(tweets)

In [19]:
terms = tfidf_vectorizer.get_feature_names()

In [7]:
# terms

In [21]:
tfidf_matrix.shape

(65499, 11251)

In [37]:
km = KMeans(n_clusters=100)

In [38]:
km.fit_transform(tfidf_matrix)

array([[ 1.12452538,  1.10347131,  1.04802324, ...,  1.13941057,
         1.0383493 ,  0.70750375],
       [ 1.12343891,  1.10353445,  1.03626849, ...,  1.13941057,
         0.9864526 ,  1.2224683 ],
       [ 1.12249743,  1.10321122,  1.04732999, ...,  1.13829558,
         1.03883599,  1.2224683 ],
       ..., 
       [ 1.12452538,  1.10363403,  1.04810103, ...,  1.13696802,
         1.03863898,  1.2224683 ],
       [ 1.11373546,  1.1035651 ,  1.04798429, ...,  1.13791088,
         1.03876922,  1.2224683 ],
       [ 1.12319521,  1.10363403,  1.04773422, ...,  1.13693713,
         1.03875732,  1.2224683 ]])

In [39]:
clusters = km.labels_.tolist()

In [40]:
users = list(data["Nickname"])

In [41]:
tweet_data = { 'Nickname': users, 'tweet': tweets, 'cluster': clusters}

frame = pd.DataFrame(tweet_data, index = [clusters] , columns = ['Nickname', 'tweet', 'cluster'])

In [42]:
frame

,Nickname,tweet,cluster
99,BillSchulhoff,#EmergencyMedicine alert: Patient Care Associa...,99
98,danipolis,"Buen día, ya listo para irme al #gym \n\nPrínc...",98
4,KJacobs27,Egg in a muffin!!! (@ Rocket Baby Bakery - @ro...,4
4,stncurtis,Egg in a muffin!!! (@ Rocket Baby Bakery - @ro...,4
4,wi_borzo,Egg in a muffin!!! (@ Rocket Baby Bakery - @ro...,4
4,KirstinMerrell,Egg in a muffin!!! (@ Rocket Baby Bakery - @ro...,4
4,Jkosches86,Egg in a muffin!!! (@ Rocket Baby Bakery - @ro...,4
57,tmj_pa_retail,Interested in a #Transportation #job near #Ced...,57
4,Vonfandango,Egg in a muffin!!! (@ Rocket Baby Bakery - @ro...,4
17,tmj_bal_jobs,#Engineering #Job alert: Prod Engr - Software ...,17


In [43]:
frame['cluster'].value_counts()

4     37071
49     1839
98     1636
63     1273
36     1130
50     1095
2      1077
46     1059
1      1012
64      661
77      642
3       597
73      564
47      548
72      537
51      508
14      481
25      475
38      459
79      450
85      445
86      437
53      419
54      404
22      403
57      401
75      387
60      371
67      369
17      364
      ...  
59       71
37       62
61       61
74       50
82       45
48       43
16       42
97       41
76       36
12       36
0        32
40       29
71       28
91       26
81       23
68       21
65       21
66       20
52       19
32       19
11       18
23       16
89       11
94        9
62        8
13        8
44        7
80        7
43        5
20        2
Name: cluster, dtype: int64

In [44]:
num_clusters = 100

In [45]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
print(order_centroids)

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print() #add whitespace
    print() #add whitespace
    
#     print("Cluster %d titles:" % i, end='')
#     for title in frame.ix[i]['Nickname'].values.tolist():
#         print(' %s,' % title, end='')
#     print() #add whitespace
#     print() #add whitespace
    
print()
print()

In [26]:
order_centroids

NameError: name 'order_centroids' is not defined

In [ ]:
import string
import collections
 
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

In [ ]:
def process_text(text, stem=True):
    """ Tokenize text and stem words removing punctuation """
    text = text.translate(None, string.punctuation)
    tokens = word_tokenize(text)
 
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens
 
def cluster_texts(texts, clusters=3):
    """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
    vectorizer = TfidfVectorizer(tokenizer=process_text,
                                 stop_words=stopwords.words('english'),
                                 max_df=0.5,
                                 min_df=0.1,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    km_model = KMeans(n_clusters=clusters)
    km_model.fit(tfidf_model)
 
    clustering = collections.defaultdict(list)
 
    for idx, label in enumerate(km_model.labels_):
        clustering[label].append(idx)
 
    return clustering
 
 


In [ ]:
clusters = cluster_texts(data[["Nickname", "Tweet content"]], 7)
print(dict(clusters))

In [ ]:
tweets

In [ ]:
data[["Nickname", "Tweet content"]]

In [ ]:
data.columns

In [46]:
# Authors: Gael Varoquaux, Nelle Varoquaux
# License: BSD 3 clause

import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

In [ ]:
knn_graph = kneighbors_graph(tfidf_matrix, 30, include_self=False)


In [ ]:
for connectivity in (None, knn_graph):
    for n_clusters in (30, 3):
        plt.figure(figsize=(10, 4))
        for index, linkage in enumerate(('average', 'complete', 'ward')):
            plt.subplot(1, 3, index + 1)
            model = AgglomerativeClustering(linkage=linkage,
                                            connectivity=connectivity,
                                            n_clusters=n_clusters)
            t0 = time.time()
            model.fit(X)
            elapsed_time = time.time() - t0
            plt.scatter(X[:, 0], X[:, 1], c=model.labels_,
                        cmap=plt.cm.spectral)
            plt.title('linkage=%s (time %.2fs)' % (linkage, elapsed_time),
                      fontdict=dict(verticalalignment='top'))
            plt.axis('equal')
            plt.axis('off')

            plt.subplots_adjust(bottom=0, top=.89, wspace=0,
                                left=0, right=1)
            plt.suptitle('n_cluster=%i, connectivity=%r' %
                         (n_clusters, connectivity is not None), size=17)


plt.show()